# Question 1: Performance of schools with art programs versus no art programs

Questions associated with art programs vs. no art programs
2018-2019 Q10 "Which of the following art disciplines did you offer at your school this year (check all that apply)?"
2017-2018 Q10 "Which of the following art disciplines did you offer at your school this year (check all that apply)?"
2016-2017 Q10 "Which of the following art disciplines did you offer at your school this year (check all that apply)?"

In [1]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [21]:
#load data 
#resource for converting:https://medium.com/better-programming/using-python-to-convert-worksheets-in-an-excel-file-to-separate-csv-files-7dd406b652d7

#survey_2014_2015 = "2014-2015_Arts_Survey_Data.csv"
#survey_2015_2016 = "2015-2016_Arts_Survey_Data.csv"
survey_2016_2017 = "2016_-_2017_Arts_Survey_data.csv"
survey_2017_2018 = "2017-2018_Arts_Data_Report (1).csv"
survey_2018_2019 = "Copy of 2018 -2019 Arts Data Report.csv"
ELA_2013_2019 = "2013-2019 ELA State Test Results.csv"
Math_2013_2019 = "2013-2019 Math State Test Results.csv"

#survey_2014_2015_df = pd.read_csv(survey_2014_2015, encoding='utf-8', dtype = 'unicode')
#survey_2015_2016_df = pd.read_csv(survey_2015_2016, encoding='utf-8', dtype = 'unicode')
survey_2016_2017_df = pd.read_csv(survey_2016_2017, encoding='utf-8', dtype = 'unicode')
survey_2017_2018_df = pd.read_csv(survey_2017_2018, encoding='utf-8', dtype = 'unicode')
survey_2018_2019_df = pd.read_csv(survey_2018_2019, encoding='utf-8', dtype = 'unicode')
ELA_2013_2019_df = pd.read_csv(ELA_2013_2019, encoding='utf-8', dtype = 'unicode')
Math_2013_2019_df = pd.read_csv(Math_2013_2019, encoding='utf-8', dtype = 'unicode')
Math_2013_2019_df.head(1)

,Unnamed: 0,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M01532013All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,278,16,59.3,11,40.7,0,0.0,0,0.0,0,0.0


In [23]:
#Merge data by DBN and SchoolYear


#Add SchoolYear to each survey file
survey_2016_2017_df["Year"] = '2017'
survey_2017_2018_df["Year"] = '2018'
survey_2018_2019_df["Year"] = '2019'

#Add subject to state test score files
ELA_2013_2019_df["Subject"] = 'ELA'
Math_2013_2019_df["Subject"] = 'Math'

#Change Q0_DBN column to just DBN in survey data
survey_2016_2017_dbn = survey_2016_2017_df.rename(columns = {'Q0_DBN':'DBN'})
survey_2016_2017_dbn
survey_2017_2018_dbn = survey_2017_2018_df.rename(columns = {'Q0_DBN':'DBN'})
survey_2017_2018_dbn
survey_2018_2019_dbn = survey_2018_2019_df.rename(columns = {'Q0_DBN':'DBN'})
survey_2018_2019_dbn

#Append survey results
survey_2016_2018 = survey_2016_2017_dbn.append(survey_2017_2018_dbn)
survey_2016_2019 = survey_2016_2018.append(survey_2018_2019_dbn)
survey_2016_2019
#append state test files
TestResults_2013_2019 = ELA_2013_2019_df.append(Math_2013_2019_df)
TestResults_2013_2019

#Merge survey and state test results files
combined_df = pd.merge(TestResults_2013_2019, survey_2016_2019, how='inner', on=['Year', 'DBN'])
combined_df


,Unnamed: 0,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,...,Q9_20,Q9_21,Q9_22,Q9_3,Q9_4,Q9_5,Q9_6,Q9_7,Q9_8,Q9_9
0,01M01532017All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2017,All Students,27,302,10,37.0,...,1,0,0,0,0,1,1,1,1,0
1,01M01542017All Students,01M015,P.S. 015 ROBERTO CLEMENTE,4,2017,All Students,23,314,1,4.3,...,1,0,0,0,0,1,1,1,1,0
2,01M01552017All Students,01M015,P.S. 015 ROBERTO CLEMENTE,5,2017,All Students,17,310,1,5.9,...,1,0,0,0,0,1,1,1,1,0
3,01M015All Grades2017All Students,01M015,P.S. 015 ROBERTO CLEMENTE,All Grades,2017,All Students,67,308,12,17.9,...,1,0,0,0,0,1,1,1,1,0
4,01M01532017All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2017,All Students,29,302,8,27.6,...,1,0,0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27170,32K562All Grades2019All Students,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2019,All Students,323,593,113,35.0,...,0,0,0,0,0,0,0,0,0,0
27171,32K56262019All Students,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,6,2019,All Students,108,588,56,51.9,...,0,0,0,0,0,0,0,0,0,0
27172,32K56272019All Students,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,7,2019,All Students,116,590,57,49.1,...,0,0,0,0,0,0,0,0,0,0
27173,32K56282019All Students,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,8,2019,All Students,108,595,53,49.1,...,0,0,0,0,0,0,0,0,0,0


#explore, clean, and merge data

-need to clean school years so that you can merge on school code and school year
-need to use data dictionary to figure out what the survey respondant answered for each question 

In [38]:
#look at columns of data
#1819
    #Q10_R1_C1 = Dance, Yes
    #Q10_R1_C2 = Dance, No
    #Q10_R2_C1 = Film, Yes
    #Q10_R2_C2 = Film, No
    #Q10_R3_C1 = Music, Yes
    #Q10_R3_C2 = Music, No
    #Q10_R4_C1 = Theater, Yes
    #Q10_R4_C2 = Theater, No
    #Q10_R5_C1 = Visual Arts, Yes
    #Q10_R5_C2 = Visual Arts, No
Q10_1819columns = [col for col in survey_2018_2019_df if col.startswith(' Q10')]

test = (survey_2018_2019_df[Q10_1819columns])
test.head(3)



,Q10_R1_C1,Q10_R1_C2,Q10_R2_C1,Q10_R2_C2,Q10_R3_C1,Q10_R3_C2,Q10_R4_C1,Q10_R4_C2,Q10_R5_C1,Q10_R5_C2
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0


In [68]:
#look at columns of data
#1819
    #Q10_R1_C1 = Dance, Yes
    #Q10_R1_C2 = Dance, No
    #Q10_R2_C1 = Film, Yes
    #Q10_R2_C2 = Film, No
    #Q10_R3_C1 = Music, Yes
    #Q10_R3_C2 = Music, No
    #Q10_R4_C1 = Theater, Yes
    #Q10_R4_C2 = Theater, No
    #Q10_R5_C1 = Visual Arts, Yes
    #Q10_R5_C2 = Visual Arts, No
col_rename_dict = {
    " Q10_R1_C1": "Dance_Yes",
    " Q10_R1_C2": "Dance_No",
    " Q10_R2_C1": "Film_Yes",
    " Q10_R2_C2": "Film_No",
    " Q10_R3_C1": "Music_Yes",
    " Q10_R3_C2": "Music_No",
    " Q10_R4_C1": "Theater_Yes",
    " Q10_R4_C2": "Theater_No",
    " Q10_R5_C1": "VisualArts_Yes",
    " Q10_R5_C2": "VisualArts_No"
}
renamed_df = test.rename(columns=col_rename_dict)
renamed_df



,Dance_Yes,Dance_No,Film_Yes,Film_No,Music_Yes,Music_No,Theater_Yes,Theater_No,VisualArts_Yes,VisualArts_No
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1541,1,0,0,1,1,0,1,0,1,0
1542,1,0,1,0,1,0,1,0,1,0
1543,1,0,0,1,1,0,0,1,1,0
1544,1,0,0,1,0,1,0,1,1,0


In [69]:
#create a column that sums up all the Yes columns
renamed_df['Dance_Yes'] = renamed_df['Dance_Yes'].astype(float)
renamed_df['Dance_No'] = renamed_df['Dance_No'].astype(float)
renamed_df['Film_Yes'] = renamed_df['Film_Yes'].astype(float)
renamed_df['Film_No'] = renamed_df['Film_No'].astype(float)
renamed_df['Music_Yes'] = renamed_df['Music_Yes'].astype(float)
renamed_df['Music_No'] = renamed_df['Music_No'].astype(float)
renamed_df['Theater_Yes'] = renamed_df['Theater_Yes'].astype(float)
renamed_df['Theater_No'] = renamed_df['Theater_No'].astype(float)
renamed_df['VisualArts_Yes'] = renamed_df['VisualArts_Yes'].astype(float)
renamed_df['VisualArts_No'] = renamed_df['VisualArts_No'].astype(float)


renamed_df["Sum of Programs"] = renamed_df.sum(axis=1)
renamed_df.sort_values(by=['Sum of Programs'], ascending =True)

,Dance_Yes,Dance_No,Film_Yes,Film_No,Music_Yes,Music_No,Theater_Yes,Theater_No,VisualArts_Yes,VisualArts_No,Sum of Programs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1511,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,5.0
1512,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0
1513,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,5.0
1515,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,5.0


In [46]:
#analyze data

In [ ]:
#visualize data